In [1]:
from spacy import displacy
import pandas as pd
import json
from pandas import DataFrame
import spacy
import sys, json
import random

In [2]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
#     try:
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f:
        lines = f.readlines()
    count_line = 0
    for line in lines:
        data = json.loads(line)
        text = data['content']
        entities = []
#         return data
        if not data['annotation']:
            continue
        for annotation in data['annotation']['annotationResult']:
            #only a single point in text annotation.
            try:
                point = annotation['points'][0]
            except:
                print(count_line)
                print(data['annotation'])
            labels = annotation['label']
            # handle both list of labels or a single label.
            if not isinstance(labels, list):
                labels = [labels]

            for label in labels:
                #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                entities.append((point['start'], point['end'] + 1 ,label))
        count_line +=1
        training_data.append((text, {"entities" : entities}))

    return training_data
#     except Exception as e:
#         logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
#         return None

In [4]:
def train_spacy(file_path):
    TRAIN_DATA = convert_dataturks_to_spacy(file_path);
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)

    # add labels
    try:
        for _, annotations in TRAIN_DATA:
            for ent in annotations.get('entities'):
                ner.add_label(ent[2])
    except:
        print("missing")

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    
    #do prediction
    doc = nlp("Samsing mobiles below $100")
    return nlp

In [5]:
FILE_COMPLETE = "dump.json"
nlp_ = train_spacy(FILE_COMPLETE)

Statring iteration 0
{'ner': 7880.578515010709}
Statring iteration 1
{'ner': 5572.440920216742}
Statring iteration 2
{'ner': 4841.791426243034}
Statring iteration 3
{'ner': 4155.274966260941}
Statring iteration 4
{'ner': 3944.6194341536507}
Statring iteration 5
{'ner': 3584.7441854258504}
Statring iteration 6
{'ner': 3210.4040615635345}
Statring iteration 7
{'ner': 3133.551317985392}
Statring iteration 8
{'ner': 2863.9323135290497}
Statring iteration 9
{'ner': 2492.5375070037676}


In [16]:
doc = nlp_("I'm not a believer but I would like to know more about Jesus!")
displacy.render(doc, style='ent', jupyter=True)

In [17]:
nlp_.to_disk('evangelism_model')